In [1]:
import pandas as pd

# Specify the path to your uploaded CSV file
csv_path = 'FullOutput.csv'

# Read the CSV file using pandas
df = pd.read_csv(csv_path)

# Now you can work with the dataframe 'df'
# ...rest of your code...

In [2]:
df

,TAGNAME,TIME,SUNANGLE,VALUE,Latitude,Longitude,Temp 2m,Dewpoint 2m,Pressure,Precipitation,GHI,DHI,BNI,Location,Date,Time,Date.1,hours
0,Main Optic,2021-06-30 02:59:21,1.235129,0,60.741,-0.795,10.739375,7.730220,101943.76110,1.241418e-08,51.624280,51.624280,0.000000,Balta_Sound,2021-06-30 00:00:00,02:59:21,2023-06-30 00:00:00,2.989167
1,Main Optic,2021-06-30 22:02:12,-1.404698,1,60.741,-0.795,11.618693,9.279538,101979.90700,0.000000e+00,0.000000,0.000000,0.000000,Balta_Sound,2021-06-30 00:00:00,22:02:12,2023-06-30 00:00:00,22.036667
2,ML1 Status,2021-06-30 04:05:02,-1.652963,0,49.868,-6.435,14.996952,12.833996,101844.88250,0.000000e+00,11.123197,9.471215,8.955114,BishopRock,2021-06-30 00:00:00,04:05:02,2023-06-30 00:00:00,4.083889
3,ML1 Status,2021-06-30 21:03:05,-2.452919,1,49.868,-6.435,15.907148,14.062195,101988.51160,0.000000e+00,0.000000,0.000000,0.000000,BishopRock,2021-06-30 00:00:00,21:03:05,2023-06-30 00:00:00,21.051389
4,ML1 Status,2021-06-30 04:14:05,0.698762,0,51.200,-4.202,13.560559,13.101573,101295.95810,9.797729e-08,33.311705,24.922714,42.892696,BullPointPost,2021-06-30 00:00:00,04:14:05,2023-06-30 00:00:00,4.234722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54301,ML1 Status,2016-01-03 16:20:07,-0.076861,1,51.200,-4.202,9.776766,6.886808,97809.60002,5.113424e-04,0.290351,0.290351,0.000000,BullPointPre,2016-01-03 00:00:00,16:20:07,2018-01-03 00:00:00,16.335278
54302,ML1 Status,2016-01-02 08:45:13,2.372444,0,51.200,-4.202,10.515775,9.588061,98698.45236,1.221071e-04,48.518186,48.518186,0.000000,BullPointPre,2016-01-02 00:00:00,08:45:13,2018-01-02 00:00:00,8.753611
54303,ML1 Status,2016-01-02 15:43:26,3.755937,1,51.200,-4.202,9.691194,8.532271,99582.84903,3.307044e-04,0.098296,0.098296,0.000000,BullPointPre,2016-01-02 00:00:00,15:43:26,2018-01-02 00:00:00,15.723889
54304,ML1 Status,2016-01-01 09:11:30,5.166971,0,51.200,-4.202,7.969061,4.444422,101017.98590,5.429240e-05,53.031738,53.031738,0.000000,BullPointPre,2016-01-01 00:00:00,09:11:30,2018-01-01 00:00:00,9.191667


In [6]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 33.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 43.9 MB/s eta 0:00:00


In [12]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.opflow import Z, I, StateFn
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [13]:
# Define the features
features = ['SUNANGLE', 'Temp 2m', 'GHI', 'DHI', 'hours']

# Extract the features and labels
X = df[features].values
y = df['VALUE'].values

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Perform feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# Define the quantum feature map
num_qubits = len(features)
depth = 3
entanglement = 'full'
feature_map = ZZFeatureMap(num_qubits, reps=depth, entanglement=entanglement)

In [18]:
backend = Aer.get_backend('qasm_simulator')

# Define the quantum instance
quantum_instance = QuantumInstance(backend)

# Define the QSVM algorithm
qsvm = SVC()

# Train the QSVM algorithm
qsvm.fit(X_train_scaled, y_train)

SVC()

In [19]:
X_train_scaled

array([[-0.39557958, -0.72594636, -0.53621992, -0.60659715,  0.89729077],
       [-0.15152538, -0.26779881, -0.48167974, -0.52477392,  0.81304232],
       [-1.20781071, -1.15458254, -0.53621992, -0.60659715,  1.14260914],
       ...,
       [-0.37686368,  1.38452053, -0.29664289, -0.29231076, -1.35910056],
       [ 1.48107939,  0.89332531,  1.59540858,  2.59135212, -0.94273369],
       [-0.36651399, -0.47304681, -0.53621992, -0.60659715,  1.0509337 ]])

In [20]:
y_pred_train = qsvm.predict(X_train_scaled)
y_pred_test = qsvm.predict(X_test_scaled)

In [22]:
y_pred_train

array([1, 1, 1, ..., 0, 0, 1])

In [23]:
# Evaluate the accuracy
accuracy_train = np.mean(y_pred_train == y_train)
accuracy_test = np.mean(y_pred_test == y_test)

# Print the accuracies
print("Training Accuracy:", accuracy_train)
print("Test Accuracy:", accuracy_test)

Training Accuracy: 0.9269404290580978
Test Accuracy: 0.919075676670963
